# DSC 630 - Course Project
## Python Data Initial Clean Up Code

In [1]:
import pandas as pd
import numpy as np
import xlrd
import os
import re


In [2]:
#read in main files
wb = xlrd.open_workbook("Data\AnimalIntakeWithResultsExtended_2015_2016.xlsx", logfile=open(os.devnull, 'w'))
data20152016 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalIntakeWithResultsExtended_2017_2018.xlsx", logfile=open(os.devnull, 'w'))
data20172018 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalIntakeWithResultsExtended_2019.xlsx", logfile=open(os.devnull, 'w'))
data2019 = pd.read_excel(wb, engine='xlrd')

#concatenate the datasets
data_df = pd.concat([data20152016, data20172018, data2019])

del data20152016, data20172018, data2019

data_df.head()

,Animal #,ARN,Animal Name,Animal Type,Species,Primary Breed,Secondary Breed,Distinguishing Markings,Gender,Altered,...,Outcome Agency Name,Agency Street Address,Agency Unit Number,Agency City,Agency Province,Agency Postal Code,Agency Email,Agency Home Phone,Agency Cell Number,RN
0,A10338615,LA54918A,Chaz,Dog,Dog,Chow Chow,Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,A10345450,LA54552A,Snowball,Dog,Dog,"Poodle, Miniature",Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,A10357120,LA57609A,Louise (Lu),Cat,Cat,Domestic Shorthair,Domestic Shorthair,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,A10363334,LA54952A,Lucky,Dog,Dog,"Poodle, Miniature",Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,A10363778,LA34139,Zoey,Cat,Cat,Domestic Medium Hair,Domestic Shorthair,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [3]:
#save merged file
data_df.to_csv('AllIntakeWithResults.csv', index = False)

In [4]:
data_df.shape

(12729, 87)

In [5]:
data_df.columns

Index(['Animal #', 'ARN', 'Animal Name', 'Animal Type', 'Species',
       'Primary Breed', 'Secondary Breed', 'Distinguishing Markings', 'Gender',
       'Altered', 'Danger', 'Danger Reason', 'Date Of Birth',
       'Age in Months Intake', 'Age Group', 'Intake Asilomar Status',
       'Intake Condition', 'Intake Record Owner', 'Intake Date', 'Intake Type',
       'Intake Subtype', 'Found Address', 'Found Zip Code', 'Reason',
       'Intake SiteName', 'Jurisdiction In', 'Agency Name', 'Agency Member',
       'Agency Member Phone', 'Agency Address', 'Intake Person ID',
       'Intake Person ID Type', 'Intake Person #', 'Admitter',
       'Street Address', 'Unit Number', 'City', 'Province', 'Postal Code',
       'Admitter's Email', 'Admitter's Home Phone', 'Admitter's Cell Phone',
       'Initial Stage', 'Initial Review Date', 'Age in Months Current',
       'Microchip Issue Date', 'Microchip Provider', 'Microchip Number',
       'Pet ID', 'Pet ID Type', 'Status', 'Stage', 'Location', 'Su

In [6]:
#check the difference between type and species
data_df['Species'].unique(), data_df['Species'].notna().sum(), data_df['Species'].describe()

(array(['Dog', 'Cat', 'Rabbit', 'Ferret', 'Lizard', 'Pig', 'Peacock',
        'Duck', 'Guinea Pig', 'Chicken', 'Hamster', 'Dove', 'Goat',
        'Parakeet', 'Turkey', 'Fowl', 'Cockatoo', 'Rat', 'Parrot',
        'Turtle', 'Horse', 'Cockatiel', 'Cattle (Cow)', 'Iguana'],
       dtype=object),
 12729,
 count     12729
 unique       24
 top         Dog
 freq       7235
 Name: Species, dtype: object)

In [7]:
data_df['Animal Type'].unique(), data_df['Animal Type'].notna().sum(), data_df['Animal Type'].describe()

(array(['Dog', 'Cat', 'Domestic', 'Reptile', 'Livestock', 'Wild Bird',
        'Rodent', 'Bird'], dtype=object),
 12729,
 count     12729
 unique        8
 top         Dog
 freq       7235
 Name: Animal Type, dtype: object)

In [8]:
#since we are only concerned with Dogs and cats both of these fields aren't needed.  I'm going to restrict the data
#to only dogs and cats
data_df = data_df[(data_df['Species'] == 'Dog') |  (data_df['Species'] == 'Cat')]
data_df.shape

(12412, 87)

In [9]:
# we need to exclude any animals who were adopted after 2/29/2020 to avoid future leakage.
data_df['Outcome Date'] = pd.to_datetime(data_df['Outcome Date'])
data_df = data_df[data_df['Outcome Date'] < pd.Timestamp(2020, 3, 1)]

In [10]:
#create clean dataset
clean_data = pd.DataFrame()
clean_data['AnimalID'] = data_df['Animal #']
clean_data['Species'] = data_df['Species']

In [11]:
data_df['Primary Breed'].unique(), data_df['Primary Breed'].notna().sum(), data_df['Primary Breed'].describe()

(array(['Chow Chow', 'Poodle, Miniature', 'Domestic Shorthair',
        'Domestic Medium Hair', 'Retriever, Labrador',
        'Terrier, Yorkshire', 'Terrier, Pit Bull',
        'Spaniel, American Cocker', 'Spaniel', 'Cornish Rex', 'Hound',
        'Manx', 'Terrier', 'Chihuahua, Short Coat', 'Terrier, Rat',
        'Dachshund, Standard Smooth Haired', 'Shih Tzu', 'Bullmastiff',
        'Australian Cattle Dog', 'Bulldog, American', 'Maltese',
        'Dachshund, Miniature Smooth Haired', 'Griffon, Brussels',
        'Basenji', 'Retriever', 'Pekingese', 'Shepherd', 'Boxer',
        'Chihuahua, Long Coat', 'Terrier, Airedale', 'German Shepherd',
        'Lhasa Apso', 'Siamese', 'Terrier, American Pit Bull', 'Beagle',
        'Schnauzer, Miniature', 'Domestic Longhair',
        'Mixed Breed, Large (over 44 lbs fully grown)', 'Pomeranian',
        'Alaskan Husky', 'Catahoula Leopard dog', 'Welsh Corgi, Pembroke',
        'Border Collie', 'Rhodesian Ridgeback', 'Terrier, Boston',
        'Po

In [12]:
data_df['Secondary Breed'].unique(), data_df['Secondary Breed'].notna().sum(), data_df['Secondary Breed'].describe()

(array(['Mix', 'Domestic Shorthair', 'Boxer', nan, 'Welsh Corgi, Cardigan',
        'Retriever', 'Shepherd', 'Terrier, Jack Russell',
        'Bulldog, American', 'Terrier, Soft Coated Wheaten',
        'Poodle, Miniature', 'Affenpinscher', 'Terrier', 'Chow Chow',
        'Terrier, Boston', 'Bulldog', 'Basset Hound',
        'Rhodesian Ridgeback', 'Maltese', 'Miniature Pinscher',
        'Terrier, Yorkshire', 'Terrier, Pit Bull', 'Hound',
        'Chihuahua, Short Coat', 'Dachshund, Standard Smooth Haired',
        'Rottweiler', 'Retriever, Labrador', 'Italian Greyhound',
        'Terrier, Scottish', 'Shih Tzu', 'Collie, Smooth',
        'Terrier, Tibetan', 'Siamese', 'Terrier, Staffordshire Bull',
        'Retriever, Golden', 'Chihuahua, Long Coat', 'Siberian Husky',
        'Papillon', 'Dachshund, Miniature Smooth Haired', 'Dalmatian',
        'Chinese Crested', 'Spaniel', 'Collie, Rough', 'Beagle',
        'Australian Cattle Dog', 'Bichon Frise', 'Pug',
        'Catahoula Leopard do

In [13]:
clean_data['PrimaryBreed'] = data_df['Primary Breed']
clean_data['SecondaryBreed'] = data_df['Secondary Breed']

In [14]:
#view data contained in distinguishing markings
data_df['Distinguishing Markings'].unique(), data_df['Distinguishing Markings'].notna().sum(), data_df['Distinguishing Markings'].describe()

#only 374 records have values, not enough for use in a model.

(array([nan, 'SEVERELY MATTED!', 'wearing Thundershirt when found',
        'missing hair along back',
        'Came in with end of tail injured...possibly bitten off',
        'wound to neck, possibly a cat fight wound',
        'longer coat, hair in ears',
        'white spot on lower belly, white hairs on chest',
        'few white hairs on belly, no white on chest', 'spot on nose',
        'front leg injury, possible HBC', 'cataracts/eye discharge',
        'darker of two', 'Brown head, white/grey back',
        'light brown spots on back', 'black/brindle spots on back',
        'Largest of litter', 'Smallest of litter', 'Green eyes',
        'yellow eyes', 'Runny eyes', 'white with black spot on the head',
        'more white on face', 'hairloss and scabbing on tail, paws',
        'MATTED COAT', 'WHITE SOCKS',
        'FINDER CLIPPED DOG&#39;S HAIR/ Right eye is cloudy ',
        'half of tail missing, appears healed', 'lazy eye', 'Three legs',
        'White stripe on nose', 'le

In [15]:
#gender will definitely be used but lets see if any are not M or F
data_df['Gender'].describe(), data_df['Gender'].unique()

(count     12372
 unique        3
 top           M
 freq       6198
 Name: Gender, dtype: object,
 array(['M', 'F', 'U'], dtype=object))

In [16]:
clean_data['Gender'] = data_df['Gender']

In [17]:
#all animals should be altered at the outcome
data_df['Altered'].describe(), data_df['Altered'].unique()

(count     12372
 unique        2
 top         Yes
 freq      12326
 Name: Altered, dtype: object,
 array(['Yes', 'No'], dtype=object))

In [18]:
#how many are no
data_df[data_df['Altered'] == 'No']
#only 46 - not enough to perform meaningful evaluation on this factor.  i'm more interested in whether they were altered at
#intake which I will have to get separately.

,Animal #,ARN,Animal Name,Animal Type,Species,Primary Breed,Secondary Breed,Distinguishing Markings,Gender,Altered,...,Outcome Agency Name,Agency Street Address,Agency Unit Number,Agency City,Agency Province,Agency Postal Code,Agency Email,Agency Home Phone,Agency Cell Number,RN
558,A30483910,LA51671,Jax,Dog,Dog,Alaskan Malamute,NaN,NaN,M,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
739,A31638187,LA57950A,Stallone,Cat,Cat,Domestic Shorthair,Mix,NaN,M,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1008,A33254069,LA58092A,Ebony( renamed Onyx),Dog,Dog,"Poodle, Miniature",Mix,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1427,A25299664,LA54355A,Kiki,Dog,Dog,Maltese,Mix,NaN,M,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1843,A29984391,LA30263,Zoe,Dog,Dog,Bichon Frise,Mix,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1875,A30257815,LA57301A,Jamie Bond,Cat,Cat,Domestic Shorthair,Mix,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1926,A30504573,NaN,Travis,Dog,Dog,"Retriever, Labrador",NaN,NaN,M,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2336,A33282823,LA58189A,Paint( re named Kora),Cat,Cat,Domestic Shorthair,Mix,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2377,A33783596,LA58624A,Maggie,Dog,Dog,"Spaniel, English Springer",NaN,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2888,A29578826,la55670a,Parker,Cat,Cat,Domestic Shorthair,Mix,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [19]:
data_df['Danger'].unique(), data_df['Danger'].notna().sum(), data_df['Danger'].describe()

(array(['No', 'Yes'], dtype=object),
 12372,
 count     12372
 unique        2
 top          No
 freq      12323
 Name: Danger, dtype: object)

In [20]:
#how many yes
data_df[data_df['Danger'] == 'Yes']
# only 50 - not enough to be a meaningful factor in analysis - excluding this and danger reason

,Animal #,ARN,Animal Name,Animal Type,Species,Primary Breed,Secondary Breed,Distinguishing Markings,Gender,Altered,...,Outcome Agency Name,Agency Street Address,Agency Unit Number,Agency City,Agency Province,Agency Postal Code,Agency Email,Agency Home Phone,Agency Cell Number,RN
65,A24926339,LA54510A,Sarge,Dog,Dog,German Shepherd,Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1058,A33550897,LA58138A,Diego (renamed Chico),Dog,Dog,"Chihuahua, Long Coat",Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1116,A33752620,LA58383A,Addie,Dog,Dog,Shih Tzu,Mix,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1356,A24781891,LA56839,Maddie(Re-named Villet),Dog,Dog,Border Collie,Mix,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2485,A34189493,LA61634,Emory(Renamed Rudy),Cat,Cat,Domestic Shorthair,Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3369,A31916934,LA57774A,Cindy(Re-named Neige),Dog,Dog,"Poodle, Miniature",Mix,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3710,A29284677,LA61403,Krissy( re named Jinx),Cat,Cat,Domestic Shorthair,Mix,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3863,A30051691,NaN,Knockout,Dog,Dog,"Terrier, Pit Bull",Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3917,A30182759,LA52131,Cruiser,Dog,Dog,Shepherd,Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3926,A30191215,LA57175A,Henry,Cat,Cat,Domestic Shorthair,Mix,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [21]:
data_df.columns

Index(['Animal #', 'ARN', 'Animal Name', 'Animal Type', 'Species',
       'Primary Breed', 'Secondary Breed', 'Distinguishing Markings', 'Gender',
       'Altered', 'Danger', 'Danger Reason', 'Date Of Birth',
       'Age in Months Intake', 'Age Group', 'Intake Asilomar Status',
       'Intake Condition', 'Intake Record Owner', 'Intake Date', 'Intake Type',
       'Intake Subtype', 'Found Address', 'Found Zip Code', 'Reason',
       'Intake SiteName', 'Jurisdiction In', 'Agency Name', 'Agency Member',
       'Agency Member Phone', 'Agency Address', 'Intake Person ID',
       'Intake Person ID Type', 'Intake Person #', 'Admitter',
       'Street Address', 'Unit Number', 'City', 'Province', 'Postal Code',
       'Admitter's Email', 'Admitter's Home Phone', 'Admitter's Cell Phone',
       'Initial Stage', 'Initial Review Date', 'Age in Months Current',
       'Microchip Issue Date', 'Microchip Provider', 'Microchip Number',
       'Pet ID', 'Pet ID Type', 'Status', 'Stage', 'Location', 'Su

In [22]:
#a few values relate to the age - date of birth, age in months intake, age in months current.  I want age in months at intake
#and age at outcome.  Need to see if age in months current is based off of todays date or outcome date
#viewing a few fields to determine:
data_df[['Date Of Birth', 'Age in Months Intake', 'Intake Date', 'Age in Months Current', 'Outcome Date']].head()

,Date Of Birth,Age in Months Intake,Intake Date,Age in Months Current,Outcome Date
0,7/26/2003,144.0,7/30/2015 12:21 PM,203.0,2015-08-10 10:50:00
1,10/6/2002,152.0,6/16/2015 3:07 PM,213.0,2015-06-30 14:22:00
2,5/13/2007,108.0,5/31/2016 10:45 AM,158.0,2016-07-25 13:13:00
3,4/16/2008,82.0,3/12/2015 11:03 AM,147.0,2015-06-02 15:41:00
4,12/12/2006,98.0,3/6/2015 10:09 AM,163.0,2015-03-17 12:24:00


In [23]:
#age in months current is definitely based on current date.  I'm going to include the date of birth in case i want to calculate
#any more dates, intake date, age in Months Intake, and calculate the age at outcome
clean_data['DOB'] = pd.to_datetime(data_df['Date Of Birth'])
clean_data['IntakeDate'] = pd.to_datetime(data_df['Intake Date'])
clean_data['OutcomeDate'] = pd.to_datetime(data_df['Outcome Date'])
clean_data['IntakeAgeMonths'] = data_df['Age in Months Intake']
clean_data['OutcomeAgeMonths'] = (clean_data['OutcomeDate'] - clean_data['DOB']) / np.timedelta64(1, 'M')


In [24]:
clean_data.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160


In [25]:
data_df.columns

Index(['Animal #', 'ARN', 'Animal Name', 'Animal Type', 'Species',
       'Primary Breed', 'Secondary Breed', 'Distinguishing Markings', 'Gender',
       'Altered', 'Danger', 'Danger Reason', 'Date Of Birth',
       'Age in Months Intake', 'Age Group', 'Intake Asilomar Status',
       'Intake Condition', 'Intake Record Owner', 'Intake Date', 'Intake Type',
       'Intake Subtype', 'Found Address', 'Found Zip Code', 'Reason',
       'Intake SiteName', 'Jurisdiction In', 'Agency Name', 'Agency Member',
       'Agency Member Phone', 'Agency Address', 'Intake Person ID',
       'Intake Person ID Type', 'Intake Person #', 'Admitter',
       'Street Address', 'Unit Number', 'City', 'Province', 'Postal Code',
       'Admitter's Email', 'Admitter's Home Phone', 'Admitter's Cell Phone',
       'Initial Stage', 'Initial Review Date', 'Age in Months Current',
       'Microchip Issue Date', 'Microchip Provider', 'Microchip Number',
       'Pet ID', 'Pet ID Type', 'Status', 'Stage', 'Location', 'Su

In [26]:
#explore Age Group
data_df['Age Group'].unique(), data_df['Age Group'].notna().sum(), data_df['Age Group'].describe()

(array(['Adult Dog', 'Adult Cat', 'Kitten (6 months or less)',
        'Puppy (6 months or less)', nan, 'Adult Other', 'Unknown age',
        'Young Other'], dtype=object),
 12362,
 count         12362
 unique            7
 top       Adult Dog
 freq           5423
 Name: Age Group, dtype: object)

In [27]:
#explore asilormar status at intake
data_df['Intake Asilomar Status'].unique(), data_df['Intake Asilomar Status'].notna().sum(), data_df['Intake Asilomar Status'].describe()

(array(['Treatable-Manageable', 'Treatable-Rehabilitatable', 'Healthy',
        'Unhealthy/Untreatable', nan, 'Unassigned'], dtype=object),
 12354,
 count       12354
 unique          5
 top       Healthy
 freq         7948
 Name: Intake Asilomar Status, dtype: object)

In [28]:
#explore condition at intake
data_df['Intake Condition'].unique(), data_df['Intake Condition'].notna().sum(), data_df['Intake Condition'].describe()

(array(['Geriatic', 'Appears Normal', 'Skin Condition',
        'Heartworm Positive', 'Normal', 'Unknown', 'Injured/wounds',
        'Emaciated/Thin', 'Sick & Injured', 'Feral', 'Sick', 'Healthy',
        nan], dtype=object),
 12371,
 count              12371
 unique                12
 top       Appears Normal
 freq                7244
 Name: Intake Condition, dtype: object)

In [29]:
#evaluate intake categorical info
data_df['Intake Type'].unique(), data_df['Intake Type'].notna().sum(), data_df['Intake Type'].describe()

(array(['Owner/Guardian Surrender', 'Seized / Custody', 'Stray', 'Return',
        'Transfer In'], dtype=object),
 12372,
 count     12372
 unique        5
 top       Stray
 freq       5917
 Name: Intake Type, dtype: object)

In [30]:
data_df['Intake Subtype'].unique(), data_df['Intake Subtype'].notna().sum(), data_df['Intake Subtype'].describe()

(array(['Surrendered', 'Stray Confined', 'Stray Roaming',
        'Stray Bite Case', 'Public Drop Off', 'Stray Abandoned',
        'Owner Surrender-Other', 'Stray Possible Owner(ID TAG)',
        'After Hour Public Drop Off', 'TNR Mitigation',
        'Returned Adoption', 'Coalition Partner Outside',
        'Coalition Partner Inside', 'Stray Injured',
        'Return Adopt(within 30 Days)', 'Owned Seized', 'Bitecase',
        'Abandoned', 'Stray Cruelty', 'Born in Care',
        'Owner Surrender - Owner moving', "Owner Surrender-Doesn't Want",
        'Owner Surrender- Sick Owner', 'Aggression', 'Bite Case',
        'Owned Abandoned', 'Emergency Response', 'Stray', 'Owned Cruelty',
        'Cruelty', 'Owner Surrender Court Ordered', 'Owned Bite Case',
        'Return Barn Cat', 'Euthanasia Request',
        'Owned Inadequate Housing', 'Owner Surrender Bitecase',
        'Destruction of Property'], dtype=object),
 12372,
 count               12372
 unique                 37
 top       

In [31]:
#add these values to the clean dataset
clean_data['AgeGroup'] = data_df['Age Group']
clean_data['IntakeAsilomar'] = data_df['Intake Asilomar Status']
clean_data['IntakeCondition'] = data_df['Intake Condition']
clean_data['IntakeType'] = data_df['Intake Type']
clean_data['IntakeSubType'] = data_df['Intake Subtype']

In [32]:
#explore location found
data_df['Found Address'].unique(), data_df['Found Address'].notna().sum(), data_df['Found Address'].describe()
#this data can be very random because of the 6249 unique values, also the top entry is the address of the main shelter

(array([nan, '5300 Blk of Franklin Ave.', '4638 Washington', ...,
        '5700 Read Boulevard', '4314 Shell Road', '3810 Red Cypress Dr'],
       dtype=object),
 8829,
 count                     8829
 unique                    6249
 top       1700 Mardi Gras Blvd
 freq                        76
 Name: Found Address, dtype: object)

In [33]:
data_df['Found Zip Code'].unique(), data_df['Found Zip Code'].notna().sum(), data_df['Found Zip Code'].describe()
#this is numerical value, i want to look at jurisdiction too to see if it is the same as this one

(array([70131., 70114., 70122., 70125., 70118., 70115., 70117., 70126.,
        70121., 70448., 70065., 70123., 70072., 70130., 70112., 70128.,
        70053., 70113., 70124., 70119., 70001., 70127., 70062., 70058.,
        70129., 70068., 70471., 70037., 70005., 70361., 70116., 70002.,
        70070., 70056., 70363., 70094., 70461., 70006., 39466., 71111.,
        23510., 70043., 70582., 39769., 70458., 39501., 70003., 39120.,
        70148., 70403., 70041., 70092., 70460., 39530., 91040., 39638.,
        70359., 70052., 70814., 34747., 70807., 70014., 70075., 70301.,
        13601., 70087., 70454., 70433., 70754., 70032., 70420., 75038.,
        70031., 70809., 70085., 70785., 70769., 70548., 70067., 70615.,
        70172., 70806., 36509., 70427., 70344., 70789., 70047., 70036.,
        71301., 70836., 70051., 71203., 70339., 70360., 70079., 70529.,
        77584., 70445., 70808., 71360., 70401., 70364., 70816., 70586.,
        10128., 39425., 33912., 29710., 30240., 71458., 30318., 

In [34]:
data_df['Jurisdiction In'].unique(), data_df['Jurisdiction In'].notna().sum(), data_df['Jurisdiction In'].describe()

(array(['70131', '70114', '70122', '70125', '70118', '70115', '70117',
        '70126', '70121', '70448', '70065', '70123', '70072', '70130',
        '70112', '70128', '70053', '70113', '70124', '70005', '70001',
        '70127', '70119', '70062', '70058', '70129', nan, '70068', '70471',
        '70037', '70361', '70116', '70002', '70070', '70056', '70363',
        '70094', '70461', '70006', '71111', '23510', '70043', '70582',
        'MS', '70458', '39501', '70003', '39120', '70148', '70403',
        '70041', '70092', '70460', '70359', 'Unknown/Other', '70052',
        '70814', '70807', '70075', '70301', 'New York', '70087', '70454',
        '70433', '70754', '70032', '70420', 'TX', '70031', '70809',
        '70085', '70785', '70769', '70548', '70615', '70806', '70067',
        'Alabama', '70427', '70344', '70789', '70047', '70036', '71301',
        '70051', '71203', '70339', '70360', '70079', '70529', '70445',
        '70808', '70401', '70364', '70816', '39426', '36608', '30116',
   

In [35]:
#i like the found zip code better because it is all numeric.  I'm going to convert it to a string value and add it to the dataset
clean_data['FoundZip'] = data_df['Found Zip Code'].apply(str)

In [36]:
clean_data.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,AgeGroup,IntakeAsilomar,IntakeCondition,IntakeType,IntakeSubType,FoundZip
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,Adult Dog,Treatable-Manageable,Geriatic,Owner/Guardian Surrender,Surrendered,70131.0
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,Adult Dog,Treatable-Rehabilitatable,Appears Normal,Owner/Guardian Surrender,Surrendered,70114.0
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,Adult Cat,Healthy,Appears Normal,Owner/Guardian Surrender,Surrendered,70122.0
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,Adult Dog,Treatable-Manageable,Appears Normal,Seized / Custody,Stray Confined,70122.0
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,Adult Cat,Treatable-Manageable,Geriatic,Owner/Guardian Surrender,Surrendered,70131.0


In [37]:
data_df['Reason'].unique(), data_df['Reason'].notna().sum(),data_df['Reason'].describe()
#there are only 3000 records that have an intake reason entered.  This could be helpful though if looking only at owner
#surrenders.  Taking it to the clean data

(array(['Health of Owner / Family', 'Cannot Afford', 'Moving', nan,
        'Behavior Issues', 'Dont Want', 'Not Housebroken', 'Other',
        'Landlord Issues', 'Military Transfer of Owner', 'Unknown',
        'Health of Animal', 'No Time For Animal',
        'Pregnancy of Owner / Family', 'Unable to Train', 'Escapes',
        'Unwanted Litter', 'Adopt Return-Behavior', 'Out of Space',
        'Aggression to People', 'Adopt Return-Moving',
        'Allergic to Animal', 'Current Animal is Incompatible',
        'Adopt Return-Landlord', 'Too Active', 'No Home',
        'Aggression to Animals', 'Adopt Return-Other', 'Too Noisy',
        'Stray', 'Death of Owner / Family', 'Adopt Return-Aggression',
        'Adopt Return-Too Many', 'Adopt Return-Allergy', 'Too Big',
        'Personal Problems', 'Unrealistic Expectations',
        'Non-compliance Chap. 18', 'Too Many Animals', 'Adopt Return-Sick',
        'Destructive', 'Insurance Restrictions', 'Adopt Return-New Baby',
        'Needs Too

In [38]:
clean_data['IntakeReason'] = data_df['Reason']

In [39]:
data_df['Intake SiteName'].unique(), data_df['Intake SiteName'].notna().sum()

(array(['LA Shelter'], dtype=object), 12372)

In [40]:
#all intake was done at the main campus, this intake variable was not needed

In [41]:
#i'm not concerned with the individuals who turned in the animal or who processed the intake
#however, if the animal came from another animal welfare agency, that may be interesting to see how that affected their
#adoption
data_df['Agency Name'].unique(), data_df['Agency Name'].notna().sum(), data_df['Agency Name'].describe()

(array([nan, 'Louisiana SPCA',
        'St John The Baptist Parish Animal Shlelter',
        "My Heart's Desire Pet Adoption Center",
        'Audubon Center For Rescue of Endagered species ',
        'Terrebonne Parish Animal Shelter', 'ACRES',
        'Bossier City Animal Control', 'PETA',
        'St.Martin Parish Animal Control',
        'Humane Society of South Mississippi', 'Natchez Pet Adoptions',
        'Tangipahoa Parish Animal Control',
        'St Bernard Parish Animal Services', 'St Charles Humane Society',
        'Companion Animal Alliance', 'Livingston Parish Animal Control',
        'Plaquemines Animal Welfare Society (PAWS)',
        'City of Walker Animal Control',
        'Jefferson Parish Animal Shelter East Bank',
        'Animal Aid of Vermilion Parish',
        'Jefferson Parish Animal Shelter',
        'Calcasieu Parish Animal Services ', 'River Region Animal Rescue',
        'Used Dogs', 'LA-SPCA Special Issue',
        'St. Tammany Parish Department of Animal

In [42]:
data_df['Agency Member'].unique(), data_df['Agency Member'].notna().sum(), data_df['Agency Member'].describe()
#do not need this info

(array([nan, 'Beth Renfro', 'Samuel Counts', 'Justin Blue', 'Laura Oliver',
        'Brandi Thibodeaux', 'Travis Causey', 'Sara Dawdy',
        'Eric Durcinka', 'Linda Allen', 'Amanda Pumilia',
        'Tracy Lapeyrouse', 'Michelle Hatwood', 'Rachel Brunet',
        'Noah Harter', 'Brady Cornell', 'Anna Ware', 'Robin Menard',
        'James Carll', 'Tracy Reis', 'Kathy Fitch', 'Chip Fitz',
        'Charlotte Luna', 'Lynzee Gray', 'Leonie Agurcia', 'Lily Yap',
        'Michelle Brignac', 'Desiree Green', 'Chris BeeBee',
        'Manager Shelter', 'Jefferson Parish', 'Tracy Davenport',
        'Thaddeus Sunmoni', 'Robin Beaulieu',
        'Species Survival Center Audubon Nature Institute', 'Anne Godeaux',
        'Sherri Arias', 'Michelle Abeyta', 'Community Clinic',
        'Zachary Shupe', 'Rachel Eure', 'Tara Smith', 'Joshua Pechawer',
        'Kristen Limbert', 'Leah Atkinson', 'Michele Penny',
        'Juanita Murphy', 'Jacqie Bella', 'Andrea Peacock',
        'Joshua Critney', 'Gne

In [43]:
clean_data['IntakeAgencyAssociation'] = data_df['Agency Name']

In [44]:
data_df.columns

Index(['Animal #', 'ARN', 'Animal Name', 'Animal Type', 'Species',
       'Primary Breed', 'Secondary Breed', 'Distinguishing Markings', 'Gender',
       'Altered', 'Danger', 'Danger Reason', 'Date Of Birth',
       'Age in Months Intake', 'Age Group', 'Intake Asilomar Status',
       'Intake Condition', 'Intake Record Owner', 'Intake Date', 'Intake Type',
       'Intake Subtype', 'Found Address', 'Found Zip Code', 'Reason',
       'Intake SiteName', 'Jurisdiction In', 'Agency Name', 'Agency Member',
       'Agency Member Phone', 'Agency Address', 'Intake Person ID',
       'Intake Person ID Type', 'Intake Person #', 'Admitter',
       'Street Address', 'Unit Number', 'City', 'Province', 'Postal Code',
       'Admitter's Email', 'Admitter's Home Phone', 'Admitter's Cell Phone',
       'Initial Stage', 'Initial Review Date', 'Age in Months Current',
       'Microchip Issue Date', 'Microchip Provider', 'Microchip Number',
       'Pet ID', 'Pet ID Type', 'Status', 'Stage', 'Location', 'Su

In [45]:
#explore initial stage and initial review date
data_df['Initial Stage'].unique(), data_df['Initial Stage'].notna().sum(), data_df['Initial Stage'].describe()

(array(['New Arrival', 'Needs Routing', 'Surgery Completed',
        'Transport Pending', nan, 'Foster Program', 'Available',
        'Parvo Quarantine', 'Bite Quarantine', 'Barn Cat Adoptions',
        'Mitigation/TNR', 'Kitten Krewe', 'Fast Track', 'Surgery Request',
        'Possible Owner', 'Supervisor Hold', 'Adoption Pending',
        'Medical Treatment', 'Vet Check'], dtype=object),
 12369,
 count           12369
 unique             18
 top       New Arrival
 freq            12241
 Name: Initial Stage, dtype: object)

In [46]:
#explore initial stage and initial review date
data_df['Initial Review Date'].unique(), data_df['Initial Review Date'].notna().sum(), data_df['Initial Review Date'].describe()

(array([nan, '3/4/2018 5:00 PM', '5/24/2019 7:00 AM'], dtype=object),
 2,
 count                     2
 unique                    2
 top       5/24/2019 7:00 AM
 freq                      1
 Name: Initial Review Date, dtype: object)

In [47]:
#The initial stage looks interesting but initial review date is mostly null
clean_data['InitialStage'] = data_df['Initial Stage']

In [48]:
data_df['Pet ID'].unique()

array([nan])

In [49]:
data_df['Pet ID Type'].unique()

array([nan])

In [50]:
#all animals are released with a microchip so that information will not impact adoptions.  In addition pet id info is not used

In [51]:
# Review 'Status', 'Stage', 'Location', 'Sublocation','Outcome Asilomar Status'
data_df['Status'].unique(), data_df['Status'].notna().sum(), data_df['Status'].describe()

(array(['I', 'L', 'H', 'F'], dtype=object),
 12372,
 count     12372
 unique        4
 top           I
 freq      12140
 Name: Status, dtype: object)

In [52]:
#i'm not sure what this is and however most of the values are I so it will not be beneficial to the model

In [53]:
data_df['Stage'].unique(), data_df['Stage'].notna().sum(), data_df['Stage'].describe()

(array(['Released', 'Lost/Found', 'Licensing', 'Non Operational'],
       dtype=object),
 12372,
 count        12372
 unique           4
 top       Released
 freq         12301
 Name: Stage, dtype: object)

In [54]:
#this appears to be the final outcome for the animal.  Given that we are looking at adoptions almost all the outcomes are released

In [55]:
data_df['Status'].unique(), data_df['Status'].notna().sum(), data_df['Status'].describe()

(array(['I', 'L', 'H', 'F'], dtype=object),
 12372,
 count     12372
 unique        4
 top           I
 freq      12140
 Name: Status, dtype: object)

In [56]:
data_df['Location'].unique(), data_df['Location'].notna().sum(), data_df['Location'].describe()

(array(['Released'], dtype=object),
 12372,
 count        12372
 unique           1
 top       Released
 freq         12372
 Name: Location, dtype: object)

In [57]:
data_df['Sublocation'].unique(), data_df['Sublocation'].notna().sum(), data_df['Sublocation'].describe()

(array([nan]),
 0,
 count    0.0
 mean     NaN
 std      NaN
 min      NaN
 25%      NaN
 50%      NaN
 75%      NaN
 max      NaN
 Name: Sublocation, dtype: float64)

In [58]:
#location and sublocation are not helpful from this file, i will need to see if i can find it separately

In [59]:
data_df['Outcome Asilomar Status'].unique(), data_df['Outcome Asilomar Status'].notna().sum(), data_df['Outcome Asilomar Status'].describe()

(array(['Treatable-Manageable', 'Treatable-Rehabilitatable', 'Healthy',
        'Unhealthy/Untreatable', nan, 'Unassigned'], dtype=object),
 12371,
 count       12371
 unique          5
 top       Healthy
 freq         8000
 Name: Outcome Asilomar Status, dtype: object)

In [60]:
clean_data['OutcomeAsilomar'] = data_df['Outcome Asilomar Status']

In [61]:
data_df['Outcome Number'].unique(), data_df['Outcome Number'].notna().sum(), data_df['Outcome Number'].describe()

(array([ 8504,   809, 11226, ...,  2731,  2152,  2721], dtype=int64),
 12372,
 count    12372.000000
 mean      5188.360734
 std       3820.861794
 min          1.000000
 25%       1729.750000
 50%       4108.000000
 75%       7769.250000
 max      12286.000000
 Name: Outcome Number, dtype: float64)

In [62]:
data_df.columns

Index(['Animal #', 'ARN', 'Animal Name', 'Animal Type', 'Species',
       'Primary Breed', 'Secondary Breed', 'Distinguishing Markings', 'Gender',
       'Altered', 'Danger', 'Danger Reason', 'Date Of Birth',
       'Age in Months Intake', 'Age Group', 'Intake Asilomar Status',
       'Intake Condition', 'Intake Record Owner', 'Intake Date', 'Intake Type',
       'Intake Subtype', 'Found Address', 'Found Zip Code', 'Reason',
       'Intake SiteName', 'Jurisdiction In', 'Agency Name', 'Agency Member',
       'Agency Member Phone', 'Agency Address', 'Intake Person ID',
       'Intake Person ID Type', 'Intake Person #', 'Admitter',
       'Street Address', 'Unit Number', 'City', 'Province', 'Postal Code',
       'Admitter's Email', 'Admitter's Home Phone', 'Admitter's Cell Phone',
       'Initial Stage', 'Initial Review Date', 'Age in Months Current',
       'Microchip Issue Date', 'Microchip Provider', 'Microchip Number',
       'Pet ID', 'Pet ID Type', 'Status', 'Stage', 'Location', 'Su

In [63]:
#outcome type, outcome subtype, outcome reason
data_df['Outcome Type'].unique(), data_df['Outcome Type'].notna().sum(), data_df['Outcome Type'].describe()

(array(['Adoption'], dtype=object),
 12372,
 count        12372
 unique           1
 top       Adoption
 freq         12372
 Name: Outcome Type, dtype: object)

In [64]:
#the outcome type is the same for all since we are only looking at adoptions
data_df['Outcome Subtype'].unique(), data_df['Outcome Subtype'].notna().sum(), data_df['Outcome Subtype'].describe()

(array(['Onsite', 'Foster Home/ Fast Track', 'Off-Site PetSmart',
        'Off-Site Petco', 'Barn Cat', 'Special Event', 'Off-Site',
        'Off-Site Clearview Volunteer', 'Off-Site Jefferson Fee',
        'Off-Site Cat Practice', 'Off-Site Petco Kenner',
        'Off-Site Petco Algiers', 'Off-Site Petco Mid-City',
        'Off-Site Petco Harvey', 'Breed/Rescue',
        'Off-Site MidCity Vet.Clinic', 'Off-Site Canine Connection',
        'No Fleas Market', 'Offsite Petco Tchoup',
        'Off-site PetSmart Manhattan'], dtype=object),
 12372,
 count      12372
 unique        20
 top       Onsite
 freq        8533
 Name: Outcome Subtype, dtype: object)

In [65]:
clean_data['OutcomeSubType'] = data_df['Outcome Subtype']

In [66]:
data_df['Outcome Reason'].unique(), data_df['Outcome Reason'].notna().sum(), data_df['Outcome Reason'].describe()

(array([nan]),
 0,
 count    0.0
 mean     NaN
 std      NaN
 min      NaN
 25%      NaN
 50%      NaN
 75%      NaN
 max      NaN
 Name: Outcome Reason, dtype: float64)

In [67]:
#the reason doesn't have anything.  The other fields in this file are related to the person adopting the animal which is not 
#helpful to figuring out the biggest way to get the animals adopted.  I will still need to get some additional values from
#another file.  

In [68]:
#need to get from pure intake/outcome files
#1.prealtered
#2. size
#3.Intake location/sublocation
#4. Outcome location/sublocation
#5. Length Owned

In [69]:
#summarize the clean data set to see how many duplicates there are
#these animals were returned to the la-spca and adopted again.  


In [70]:
grouped_df = clean_data.groupby('AnimalID', as_index=False).count()[['AnimalID','IntakeDate']]
grouped_df.rename(columns={'IntakeDate':'Count'}, inplace=True)
grouped_df.sort_values("Count", ascending=False)

,AnimalID,Count
2930,A31234826,5
9144,A40288375,4
6985,A37605306,4
6223,A36646929,4
2801,A31064652,4
...,...,...
3931,A33381229,1
3932,A33382904,1
3933,A33389041,1
3934,A33389443,1


In [71]:
grouped_df[grouped_df['Count'] > 1]

,AnimalID,Count
6,A10356875,2
25,A10380452,2
32,A10383856,2
46,A13170522,2
61,A13971166,2
...,...,...
11140,A43179299,2
11164,A43201062,2
11228,A43286854,2
11238,A43302985,2


In [72]:
grouped_df[grouped_df['Count'] > 1]['Count'].sum()

1882

In [73]:
# there are only 1882 out of 12000 adoptions where animals were returned previously.  Because I need to join this data
# to other data and id's do not exist per transaction, i'm going to exclude these from the data set
#joining the data together and then filtering them out

In [74]:
#merge the grouped data to the clean set
clean_data = clean_data.merge(grouped_df, on="AnimalID", how="left")
clean_data.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,IntakeCondition,IntakeType,IntakeSubType,FoundZip,IntakeReason,IntakeAgencyAssociation,InitialStage,OutcomeAsilomar,OutcomeSubType,Count
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,Geriatic,Owner/Guardian Surrender,Surrendered,70131.0,Health of Owner / Family,NaN,New Arrival,Treatable-Manageable,Onsite,1
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,Appears Normal,Owner/Guardian Surrender,Surrendered,70114.0,Cannot Afford,NaN,New Arrival,Treatable-Rehabilitatable,Onsite,1
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,Appears Normal,Owner/Guardian Surrender,Surrendered,70122.0,Moving,NaN,New Arrival,Healthy,Foster Home/ Fast Track,1
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Appears Normal,Seized / Custody,Stray Confined,70122.0,NaN,Louisiana SPCA,New Arrival,Treatable-Manageable,Onsite,1
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,...,Geriatic,Owner/Guardian Surrender,Surrendered,70131.0,Moving,NaN,New Arrival,Treatable-Manageable,Off-Site PetSmart,1


In [75]:
clean_data.rename(columns={'Count':'TimesAdopted'}, inplace=True)
clean_data.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,IntakeCondition,IntakeType,IntakeSubType,FoundZip,IntakeReason,IntakeAgencyAssociation,InitialStage,OutcomeAsilomar,OutcomeSubType,TimesAdopted
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,Geriatic,Owner/Guardian Surrender,Surrendered,70131.0,Health of Owner / Family,NaN,New Arrival,Treatable-Manageable,Onsite,1
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,Appears Normal,Owner/Guardian Surrender,Surrendered,70114.0,Cannot Afford,NaN,New Arrival,Treatable-Rehabilitatable,Onsite,1
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,Appears Normal,Owner/Guardian Surrender,Surrendered,70122.0,Moving,NaN,New Arrival,Healthy,Foster Home/ Fast Track,1
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Appears Normal,Seized / Custody,Stray Confined,70122.0,NaN,Louisiana SPCA,New Arrival,Treatable-Manageable,Onsite,1
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,...,Geriatic,Owner/Guardian Surrender,Surrendered,70131.0,Moving,NaN,New Arrival,Treatable-Manageable,Off-Site PetSmart,1


In [76]:
clean_data = clean_data[clean_data['TimesAdopted'] == 1]
clean_data.shape

(10490, 22)

In [77]:
clean_data.dtypes

AnimalID                           object
Species                            object
PrimaryBreed                       object
SecondaryBreed                     object
Gender                             object
DOB                        datetime64[ns]
IntakeDate                 datetime64[ns]
OutcomeDate                datetime64[ns]
IntakeAgeMonths                   float64
OutcomeAgeMonths                  float64
AgeGroup                           object
IntakeAsilomar                     object
IntakeCondition                    object
IntakeType                         object
IntakeSubType                      object
FoundZip                           object
IntakeReason                       object
IntakeAgencyAssociation            object
InitialStage                       object
OutcomeAsilomar                    object
OutcomeSubType                     object
TimesAdopted                        int64
dtype: object

In [78]:
###Load Intake Only Data and get records needed


In [79]:
#read in files
wb = xlrd.open_workbook("Data\AnimalIntakeExtended2015_2016.xls", logfile=open(os.devnull, 'w'))
intake20152016 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalIntakeExtended2017_2018.xls", logfile=open(os.devnull, 'w'))
intake20172018 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalIntakeExtended2019.xls", logfile=open(os.devnull, 'w'))
intake2019 = pd.read_excel(wb, engine='xlrd')

#concatenate the datasets
intake_df = pd.concat([intake20152016, intake20172018, intake2019])

del intake20152016, intake20172018, intake2019

intake_df.head()

,Animal ID,ARN,Animal Name,Species,Primary Breed,Secondary Breed,Gender,Age,Altered,Danger,...,Source,Intake Reason,Length Owned,Unit,Injury Type,Cause,Agency Name,Location,Sub Location,Asilomar Status
0,A10323061,LA57878A,Daisy,Dog,Dalmatian,Mix,F,16yrs,No,No,...,Friend,Euthanasia Request,16.0,Years,NaN,NaN,NaN,Serenity Room,Serenity Room,Unhealthy/Untreatable
1,A10331004,134141,Hazel,Dog,"Terrier, Pit Bull",Mix,F,14yrs,No,No,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,10 Dog Healthy Hold (Sm),H-61,Treatable-Manageable
2,A10331144,la62897,Zeus,Dog,"Poodle, Toy",Mix,M,16yrs,No,No,...,NaN,NaN,0.0,NaN,NaN,NaN,Louisiana SPCA,15 ACO Unit 5,Cage 4,Unhealthy/Untreatable
3,A10336427,221260,Clouseau,Cat,Domestic Shorthair,Domestic Shorthair,M,12yrs,Yes,No,...,Friend,Other,10.0,Years,NaN,NaN,NaN,14 Shelter Clinic,14 Shelter Clinic,Unhealthy/Untreatable
4,A10336800,281446,Diamond,Dog,"Terrier, American Pit Bull",Mix,F,13yrs,Yes,No,...,NaN,NaN,0.0,NaN,NaN,NaN,Louisiana SPCA,15 ACO Unit 4,Cage 3,Treatable-Manageable


In [80]:
intake_df.shape

(27561, 42)

In [81]:
intake_df.columns

Index(['Animal ID', 'ARN', 'Animal Name', 'Species', 'Primary Breed',
       'Secondary Breed', 'Gender', 'Age', 'Altered', 'Danger',
       'Danger Reason', 'Primary Colour', 'Secondary Colour', 'Third Colour',
       'Colour Pattern', 'Second Colour Pattern', 'Size', 'Pre Altered',
       'Spayed Neutered', 'By', 'Record Owner', 'Intake Date/Time',
       'Operation Type', 'Operation Sub Type', 'Pet ID', 'Pet ID Type',
       'Location Found', 'Jurisdiction', 'Condition', 'Age Group', 'DOA',
       'Site Name', 'Source', 'Intake Reason', 'Length Owned', 'Unit',
       'Injury Type', 'Cause', 'Agency Name', 'Location', 'Sub Location',
       'Asilomar Status'],
      dtype='object')

In [82]:
#save all intakes for overal statistics
intake_df.to_csv('IntakeData.csv', index=False)

In [83]:
intake_sub = intake_df[['Animal ID', 'Intake Date/Time', 'Size', 'Length Owned', 'Unit', 'Location', 'Sub Location', 'Pre Altered']]
intake_sub = intake_sub.rename(columns={'Animal ID':'AnimalID'})
intake_sub.head()

,AnimalID,Intake Date/Time,Size,Length Owned,Unit,Location,Sub Location,Pre Altered
0,A10323061,2016-06-28 15:51:00,Medium,16.0,Years,Serenity Room,Serenity Room,N
1,A10331004,2016-03-01 12:35:00,Medium,0.0,NaN,10 Dog Healthy Hold (Sm),H-61,N
2,A10331144,2016-04-20 12:13:00,Small,0.0,NaN,15 ACO Unit 5,Cage 4,N
3,A10336427,2015-02-27 13:48:00,Small,10.0,Years,14 Shelter Clinic,14 Shelter Clinic,Y
4,A10336800,2016-09-15 14:42:00,Large,0.0,NaN,15 ACO Unit 4,Cage 3,Y


In [84]:
newdata = clean_data.merge(intake_sub, on='AnimalID', how='left')
newdata.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,OutcomeAsilomar,OutcomeSubType,TimesAdopted,Intake Date/Time,Size,Length Owned,Unit,Location,Sub Location,Pre Altered
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,Treatable-Manageable,Onsite,1,2015-07-30 12:21:00,Medium,12.0,Years,10 Dog Healthy Hold (Sm),H-63,Y
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,Treatable-Rehabilitatable,Onsite,1,2015-06-16 15:07:00,Medium,9.0,Years,10 Dog Healthy Hold (Sm),H-53,Y
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,Healthy,Foster Home/ Fast Track,1,2016-05-31 10:45:00,Small,9.0,Years,04 Cat Feral Room,CF-08,Y
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Treatable-Manageable,Onsite,1,2015-03-12 11:04:00,Small,0.0,NaN,15 ACO Unit 4,Cage 4,Y
4,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Treatable-Manageable,Onsite,1,2015-07-26 13:27:00,Small,0.0,NaN,10 Dog Healthy Hold (Sm),H-53,Y


In [85]:
newdata.shape

(10957, 29)

In [86]:
#check to see if newdata has nulls
newdata['Size'].notna().sum()

#no nulls but it looks like additional records were added

10957

In [87]:
grouped_df = newdata.groupby('AnimalID', as_index=False).count()[['AnimalID','IntakeDate']]
grouped_df.rename(columns={'IntakeDate':'Count'}, inplace=True)
grouped_df.sort_values("Count", ascending=False)

,AnimalID,Count
5873,A36843998,6
8305,A40197150,5
8185,A40031643,5
3683,A33462209,4
7240,A38763191,4
...,...,...
3599,A33318279,1
3600,A33318285,1
3601,A33318293,1
3602,A33319961,1


In [88]:
grouped_df[grouped_df['Count']>1]

,AnimalID,Count
9,A10363334,2
36,A11738206,2
47,A13625090,2
51,A13775490,2
58,A14203870,2
...,...,...
9967,A42766241,2
10069,A42917137,2
10177,A43059671,2
10223,A43131808,2


In [89]:
#investigate one of the id's in the intake sub set twice
intake_sub[intake_sub['AnimalID'] == 'A10363334']

,AnimalID,Intake Date/Time,Size,Length Owned,Unit,Location,Sub Location,Pre Altered
26,A10363334,2015-03-12 11:04:00,Small,0.0,NaN,15 ACO Unit 4,Cage 4,Y
27,A10363334,2015-07-26 13:27:00,Small,0.0,NaN,10 Dog Healthy Hold (Sm),H-53,Y


In [90]:
clean_data[clean_data['AnimalID'] == 'A10363334']

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,IntakeCondition,IntakeType,IntakeSubType,FoundZip,IntakeReason,IntakeAgencyAssociation,InitialStage,OutcomeAsilomar,OutcomeSubType,TimesAdopted
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Appears Normal,Seized / Custody,Stray Confined,70122.0,NaN,Louisiana SPCA,New Arrival,Treatable-Manageable,Onsite,1


In [91]:
#it appears that this animal ended up being returned however the outcome was not adoption so they are not listed in
#the original file.  I can create another key based on just the intake date, not time since time was sometimes different between
#seconds when I tried to match before to get the information.

In [92]:
clean_data['IntakeDateOnly'] = clean_data['IntakeDate'].dt.date
intake_sub['IntakeDateOnly'] = intake_sub['Intake Date/Time'].dt.date

In [93]:
newdata = clean_data.merge(intake_sub, on=['AnimalID', 'IntakeDateOnly'], how='left')

In [94]:
newdata.shape

(10490, 30)

In [95]:
#no new records created by the join
newdata['Size'].notna().sum()

10490

In [96]:
#no nulls created - data was merged successsfully - resetting the clean data frame to be the merged data
clean_data = newdata
del newdata

In [97]:
clean_data.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,OutcomeSubType,TimesAdopted,IntakeDateOnly,Intake Date/Time,Size,Length Owned,Unit,Location,Sub Location,Pre Altered
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,Onsite,1,2015-07-30,2015-07-30 12:21:00,Medium,12.0,Years,10 Dog Healthy Hold (Sm),H-63,Y
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,Onsite,1,2015-06-16,2015-06-16 15:07:00,Medium,9.0,Years,10 Dog Healthy Hold (Sm),H-53,Y
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,Foster Home/ Fast Track,1,2016-05-31,2016-05-31 10:45:00,Small,9.0,Years,04 Cat Feral Room,CF-08,Y
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Onsite,1,2015-03-12,2015-03-12 11:04:00,Small,0.0,NaN,15 ACO Unit 4,Cage 4,Y
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,...,Off-Site PetSmart,1,2015-03-06,2015-03-06 10:09:00,Medium,5.0,Years,04 Cat Feral Room,CF-17,Y


In [98]:
#rename some of the new fields
clean_data.rename(columns={'Length Owned':'LengthOwned', 'Unit': 'LengthOwnedUnits', 
                           'Location': 'IntakeLocation', 'Sub Location':'IntakeSubLocation', 'Pre Altered': 'PreAltered'}, inplace=True)


#drop the Intake Date/Time added by the merge
clean_data = clean_data.drop(columns='Intake Date/Time')

clean_data.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,OutcomeAsilomar,OutcomeSubType,TimesAdopted,IntakeDateOnly,Size,LengthOwned,LengthOwnedUnits,IntakeLocation,IntakeSubLocation,PreAltered
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,Treatable-Manageable,Onsite,1,2015-07-30,Medium,12.0,Years,10 Dog Healthy Hold (Sm),H-63,Y
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,Treatable-Rehabilitatable,Onsite,1,2015-06-16,Medium,9.0,Years,10 Dog Healthy Hold (Sm),H-53,Y
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,Healthy,Foster Home/ Fast Track,1,2016-05-31,Small,9.0,Years,04 Cat Feral Room,CF-08,Y
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,Treatable-Manageable,Onsite,1,2015-03-12,Small,0.0,NaN,15 ACO Unit 4,Cage 4,Y
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,...,Treatable-Manageable,Off-Site PetSmart,1,2015-03-06,Medium,5.0,Years,04 Cat Feral Room,CF-17,Y


In [99]:
clean_data['IntakeLocation'].unique(), clean_data['IntakeLocation'].notna().sum(), clean_data['IntakeLocation'].describe()

(array(['10 Dog Healthy Hold (Sm)', '04 Cat Feral Room', '15 ACO Unit 4',
        '15 ACO Unit 8', '03 Cat Healthy Hold', '15 ACO Unit 5',
        '15 ACO Unit 1', '12 Temporary Puppy Healthy Hold',
        '15 ACO Unit 2', '15 ACO Unit 6', '15 ACO Unit 3',
        '08 Dog Healthy Hold (Big)', 'St. John', '14 Shelter Clinic',
        'Puppy Healthy Hold', '05 Cat Isolation', 'Avenue Wellness Clinic',
        'Terrebonne Parish', '02 Cat Adoptions Overflow',
        '01 Puppy Health Hold', '24 Office', 'St Martins',
        '09 Dog Healthy Hold (Med)', '06 Encouragement Room',
        'Wellness Clinic', '20 Foster Care', '13 Dog Isolation',
        'St.Charles Parish', '11 Quarantine', '25 Outside Runs',
        '00 Cat Adoption Condo', '26 Private Vet Clinic', 'Petco Midcity',
        '00 Temp Grooming Rm Cat Intake', '00 Placement Condos'],
       dtype=object),
 10490,
 count                 10490
 unique                   35
 top       04 Cat Feral Room
 freq                   3214


In [100]:
clean_data['IntakeSubLocation'].unique(), clean_data['IntakeSubLocation'].notna().sum(), clean_data['IntakeSubLocation'].describe()

(array(['H-63', 'H-53', 'CF-08', 'Cage 4', 'CF-17', 'CH-28', 'CF-04',
        '10 Dog Healthy Hold (Sm)', 'Cage 5', 'Cage 2', 'H-58', 'CF-07',
        'H-56', 'H-54', 'CF-02', 'H-52', 'CT-03', 'Cage 3', 'H-55',
        'CT-04', 'CF-06', 'H-57', 'CT-02', 'Cage 1', 'H-61', 'H-60',
        'CF-12', 'CF-03', 'CT-01', 'CF-18', 'CF-01', 'CT-05',
        '08 Dog Healthy Hold (Big)', '03 Cat Healthy Hold', 'CH-21',
        'H-62', 'CF-15', 'CF-11', 'CF-13', 'CF-10', '12 Cat Temp',
        'Cage 6', 'St John', 'CH-16', 'CF-05', 'CH-42', 'CH-34', 'CH-41',
        'CH-37', 'CH-32', 'CH-33', 'CH-38', 'CH-39', 'CH-31', 'CH-35',
        '14 Shelter Clinic', 'CF-14', 'H-59', 'CH-20', 'CT-06', 'PH-03',
        'PH-04', 'PH-06', 'CI-04', 'CF-16', 'Wellness Clinic',
        'Terrebonne Parish', 'CH-24', '04 Cat Feral Room',
        '02 Cat Adoptions', 'H-01', 'H-02', '01 Puppy hh', 'CH-07',
        '24 Office', 'CH-29', 'CH-27', 'CF-09', 'CH-01', 'CA-22', 'CA-02',
        'CH-22', 'CH-36', 'CH-06', 'CH-

In [101]:
###Load Outcome Only Data and get records needed


In [102]:
#read in files
wb = xlrd.open_workbook("Data\AnimalOutcomebyIntakeExtended2015_2016.xls", logfile=open(os.devnull, 'w'))
outcome20152016 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalOutcomebyIntakeExtended2017_2018.xls", logfile=open(os.devnull, 'w'))
outcome20172018 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalOutcomebyIntakeExtended2019_2020.xls", logfile=open(os.devnull, 'w'))
outcome2019 = pd.read_excel(wb, engine='xlrd')

#concatenate the datasets
outcome_df = pd.concat([outcome20152016, outcome20172018, outcome2019])

del outcome20152016, outcome20172018, outcome2019

outcome_df.head()

,Animal #,ARN,Animal Name,Species,Primary Breed,Colors,Gender,Altered,Pre-Altered,Chip Number,...,Outcome Subtype,Outcome Site,Outcome Location,Outcome Sublocation,Outcome Jurisdiction,Transfer Out Reason,Outcome Agency,Released By,Release Date,LOS
0,A10338615,LA54918A,Chaz,Dog,Chow Chow,Black/,Male,Yes,Yes,982000362192682,...,Onsite,LA Shelter,09 Dog Healthy Hold (Med),H-44,70124,NaN,NaN,Vanessa Artigas,2015-08-10 10:50:56.000,10
1,A10345450,LA54552A,Snowball,Dog,"Poodle, Miniature",Cream/,Male,Yes,Yes,0A102E7808,...,Onsite,LA Shelter,01 Puppy Health Hold,PHH-04,70126,NaN,NaN,Greyshelneka Lewis,2015-06-30 14:22:58.077,13
2,A10357120,LA57609A,Louise (Lu),Cat,Domestic Shorthair,White/Grey/Black,Female,Yes,Yes,982000406897769,...,Foster Home/ Fast Track,LA Shelter,20 Foster Care,20 Foster Care,70119,NaN,NaN,Sheila Hernandez,2016-07-25 13:13:21.000,55
3,A10363334,LA54952A,Lucky,Dog,"Poodle, Miniature",White/,Male,Yes,Yes,4a7d4c3664,...,Onsite,LA Shelter,08 Dog Healthy Hold (Big),H-17,70113,NaN,NaN,Vanessa Artigas,2015-06-02 15:41:11.000,82
4,A10363778,LA34139,Zoey,Cat,Domestic Medium Hair,Grey/Brown,Female,Yes,Yes,4b05703d73,...,Off-Site PetSmart,LA Shelter,PetSmart N.O.,PetSmart N.O.,70115,NaN,NaN,Tonya LeJeune,2015-03-17 12:24:29.620,11


In [103]:
outcome_df.columns

Index(['Animal #', 'ARN', 'Animal Name', 'Species', 'Primary Breed', 'Colors',
       'Gender', 'Altered', 'Pre-Altered', 'Chip Number', 'Chip Provider',
       'Danger', 'Danger Reason', 'Date of Birth', 'Intake Age', 'Intake By',
       'Intake Date', 'Intake Type', 'Intake Subtype', 'Intake Reason',
       'Intake Site', 'Intake Location', 'Intake Sublocation',
       'Intake Jurisdiction', 'Intake Condition', 'Intake Agency',
       'Agency Member', 'Agency Member Phone', 'Agency Address', 'Outcome Age',
       'Outcome Age in Months', 'Outcome Date', 'Outcome By', 'Outcome Type',
       'Outcome Subtype', 'Outcome Site', 'Outcome Location',
       'Outcome Sublocation', 'Outcome Jurisdiction', 'Transfer Out Reason',
       'Outcome Agency', 'Released By', 'Release Date', 'LOS'],
      dtype='object')

In [104]:
outcome_df.shape

(13001, 44)

In [105]:
#save complete outcome data
outcome_df.to_csv("OutcomeData.csv", index=False)

In [106]:
#need to create subset of data to merge and outcome date keys to merge datasets on
outcome_sub = outcome_df[['Animal #', 'Outcome Date', 'Outcome Location', 'Outcome Sublocation']]
outcome_sub = outcome_sub.rename(columns={'Animal #':'AnimalID', 'Outcome Location':'OutcomeLocation', 
                           'Outcome Sublocation': 'OutcomeSubLocation'})

outcome_sub['OutcomeDateOnly'] = outcome_sub['Outcome Date'].dt.date
clean_data['OutcomeDateOnly'] = clean_data['OutcomeDate'].dt.date

#drop the outcome date from the subset
outcome_sub = outcome_sub.drop(columns='Outcome Date')

newdata = clean_data.merge(outcome_sub, on=['AnimalID', 'OutcomeDateOnly'], how="left")

newdata.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,IntakeDateOnly,Size,LengthOwned,LengthOwnedUnits,IntakeLocation,IntakeSubLocation,PreAltered,OutcomeDateOnly,OutcomeLocation,OutcomeSubLocation
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,2015-07-30,Medium,12.0,Years,10 Dog Healthy Hold (Sm),H-63,Y,2015-08-10,09 Dog Healthy Hold (Med),H-44
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,2015-06-16,Medium,9.0,Years,10 Dog Healthy Hold (Sm),H-53,Y,2015-06-30,01 Puppy Health Hold,PHH-04
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,2016-05-31,Small,9.0,Years,04 Cat Feral Room,CF-08,Y,2016-07-25,20 Foster Care,20 Foster Care
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,2015-03-12,Small,0.0,NaN,15 ACO Unit 4,Cage 4,Y,2015-06-02,08 Dog Healthy Hold (Big),H-17
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,...,2015-03-06,Medium,5.0,Years,04 Cat Feral Room,CF-17,Y,2015-03-17,PetSmart N.O.,PetSmart N.O.


In [107]:
newdata.shape

(10491, 32)

In [108]:
#it created one extra record
newdata['OutcomeLocation'].notna().sum()

10491

In [109]:
#no nulls
#investigate duplicate record
grouped_df = newdata.groupby('AnimalID', as_index=False).count()[['AnimalID','OutcomeDate']]
grouped_df.rename(columns={'OutcomeDate':'Count'}, inplace=True)
grouped_df.sort_values("Count", ascending=False)

,AnimalID,Count
10221,A43120087,2
0,A10336599,1
6987,A38461967,1
6989,A38462485,1
6990,A38462521,1
...,...,...
3498,A33188235,1
3499,A33191445,1
3500,A33198184,1
3501,A33199261,1


In [110]:
#view duplicated record
outcome_sub[outcome_sub['AnimalID']=='A43120087']

,AnimalID,OutcomeLocation,OutcomeSubLocation,OutcomeDateOnly
2487,A43120087,00 Cat Adoption Condo,Cat Condo,2020-02-09
2488,A43120087,20 Foster Care,20 Foster Care,2020-02-09


In [111]:
outcome_df[outcome_df['Animal #']=='A43120087'][['Animal #', 'Outcome Date']]

,Animal #,Outcome Date
2487,A43120087,2020-02-09 16:59:00
2488,A43120087,2020-02-09 12:28:00


In [112]:
outcome_df[outcome_df['Animal #']=='A43120087'][['Animal #', 'Intake Date', 'Outcome Date']]

,Animal #,Intake Date,Outcome Date
2487,A43120087,2020-02-09 15:11:45.247,2020-02-09 16:59:00
2488,A43120087,2019-10-31 11:08:30.320,2020-02-09 12:28:00


In [113]:
#THis animal was initially adopted in 2019 but was not returned until 2020.  I need to exclude any intake dates
#from before 1/1/2020.  I need to filter the original data and recreate the subset.

outcome_df = outcome_df[pd.to_datetime(outcome_df['Intake Date']) < pd.Timestamp(2020,1,1)]
outcome_df.shape

(12852, 44)

In [114]:
#need to create subset of data to merge and outcome date keys to merge datasets on
outcome_sub = outcome_df[['Animal #', 'Outcome Date', 'Outcome Location', 'Outcome Sublocation']]
outcome_sub = outcome_sub.rename(columns={'Animal #':'AnimalID', 'Outcome Location':'OutcomeLocation', 
                           'Outcome Sublocation': 'OutcomeSubLocation'})

outcome_sub['OutcomeDateOnly'] = outcome_sub['Outcome Date'].dt.date
clean_data['OutcomeDateOnly'] = clean_data['OutcomeDate'].dt.date

#drop the outcome date from the subset
outcome_sub = outcome_sub.drop(columns='Outcome Date')

newdata = clean_data.merge(outcome_sub, on=['AnimalID', 'OutcomeDateOnly'], how="left")

newdata.head()

,AnimalID,Species,PrimaryBreed,SecondaryBreed,Gender,DOB,IntakeDate,OutcomeDate,IntakeAgeMonths,OutcomeAgeMonths,...,IntakeDateOnly,Size,LengthOwned,LengthOwnedUnits,IntakeLocation,IntakeSubLocation,PreAltered,OutcomeDateOnly,OutcomeLocation,OutcomeSubLocation
0,A10338615,Dog,Chow Chow,Mix,M,2003-07-26,2015-07-30 12:21:00,2015-08-10 10:50:00,144.0,144.510611,...,2015-07-30,Medium,12.0,Years,10 Dog Healthy Hold (Sm),H-63,Y,2015-08-10,09 Dog Healthy Hold (Med),H-44
1,A10345450,Dog,"Poodle, Miniature",Mix,M,2002-10-06,2015-06-16 15:07:00,2015-06-30 14:22:00,152.0,152.794878,...,2015-06-16,Medium,9.0,Years,10 Dog Healthy Hold (Sm),H-53,Y,2015-06-30,01 Puppy Health Hold,PHH-04
2,A10357120,Cat,Domestic Shorthair,Domestic Shorthair,F,2007-05-13,2016-05-31 10:45:00,2016-07-25 13:13:00,108.0,110.443358,...,2016-05-31,Small,9.0,Years,04 Cat Feral Room,CF-08,Y,2016-07-25,20 Foster Care,20 Foster Care
3,A10363334,Dog,"Poodle, Miniature",Mix,M,2008-04-16,2015-03-12 11:03:00,2015-06-02 15:41:00,82.0,85.542733,...,2015-03-12,Small,0.0,NaN,15 ACO Unit 4,Cage 4,Y,2015-06-02,08 Dog Healthy Hold (Big),H-17
4,A10363778,Cat,Domestic Medium Hair,Domestic Shorthair,F,2006-12-12,2015-03-06 10:09:00,2015-03-17 12:24:00,98.0,99.140160,...,2015-03-06,Medium,5.0,Years,04 Cat Feral Room,CF-17,Y,2015-03-17,PetSmart N.O.,PetSmart N.O.


In [115]:
newdata.shape

(10490, 32)

In [116]:
newdata['OutcomeLocation'].notna().sum()

10490

In [117]:
#looks good.  I'm going to save the clean data in a csv to use in R for some exploratory data analysis
clean_data = newdata
clean_data.to_csv('CleanData.csv', index=False)

In [118]:
#read in and combine euthanasia stats for use in general statistics
wb = xlrd.open_workbook("Data\AnimalIntakeWithResultsExtended_Euth_2015_2016.xlsx", logfile=open(os.devnull, 'w'))
euth20152016 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalIntakeWithResultsExtended_Euth_2017_2018.xlsx", logfile=open(os.devnull, 'w'))
euth20172018 = pd.read_excel(wb, engine='xlrd')

wb = xlrd.open_workbook("Data\AnimalIntakeWithResultsExtended_Euth_2019_2020.xlsx", logfile=open(os.devnull, 'w'))
euth2019 = pd.read_excel(wb, engine='xlrd')

#concatenate the datasets
euth_df = pd.concat([euth20152016, euth20172018, euth2019])

del euth20152016, euth20172018, euth2019

euth_df.head()

,Animal #,ARN,Animal Name,Animal Type,Species,Primary Breed,Secondary Breed,Distinguishing Markings,Gender,Altered,...,Outcome Agency Name,Agency Street Address,Agency Unit Number,Agency City,Agency Province,Agency Postal Code,Agency Email,Agency Home Phone,Agency Cell Number,RN
0,A10323061,LA57878A,Daisy,Dog,Dog,Dalmatian,Mix,NaN,F,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,A10331144,la62897,Zeus,Dog,Dog,"Poodle, Toy",Mix,NaN,M,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,A10336427,221260,Clouseau,Cat,Cat,Domestic Shorthair,Domestic Shorthair,NaN,M,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,A10336800,281446,Diamond,Dog,Dog,"Terrier, American Pit Bull",Mix,NaN,F,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,A10340450,LA52217,ACE,Dog,Dog,"Terrier, Pit Bull",NaN,NaN,M,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [119]:
euth_df.shape

(5690, 87)

In [120]:
euth_df.to_csv('EuthData.csv', index=False)